In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-spam-detection/spam mail.csv
/kaggle/input/email-spam-detection/spam.csv


In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay)

In [3]:
df=pd.read_csv("/kaggle/input/email-spam-detection/spam mail.csv")

In [4]:
df.head()

,Category,Masseges
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.shape

(5572, 2)

In [6]:
df.isnull().sum()

Category    0
Masseges    0
dtype: int64

In [7]:
df["Category"].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [8]:
df["label"] = df["Category"].astype(str).str.lower().map(lambda x: 1 if "spam" in x else 0)

In [9]:
df

,Category,Masseges,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will Ì_ b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [10]:

display(df[df["Category"].astype(str).str.lower().str.contains("spam")].head(5))
display(df[df["Category"].astype(str).str.lower().str.contains("ham")].head(5))

,Category,Masseges,label
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",1


,Category,Masseges,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
6,ham,Even my brother is not like to speak with me. ...,0


In [11]:
X = df["Masseges"].astype(str)
y = df["label"].astype(int)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, stratify=df["label"], random_state=42)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents="unicode",
                             lowercase=True,
                             stop_words="english",
                             ngram_range=(1,2),
                             max_df=0.9,
                             min_df=2)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

In [14]:
pip install scikit-learn==1.4.2 imbalanced-learn==0.12.3


Note: you may need to restart the kernel to use updated packages.


In [15]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train_vec, y_train)

print("Class distribution after SMOTE:\n", pd.Series(y_train_bal).value_counts())

Class distribution after SMOTE:
 label
0    3859
1    3859
Name: count, dtype: int64


In [16]:
models = {
    "MultinomialNB": MultinomialNB(),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
}

In [17]:
results = []
for name, model in models.items():
    print("====", name, "====")
    model.fit(X_train_bal, y_train_bal)
    y_pred = model.predict(X_test_vec)
    y_proba = model.predict_proba(X_test_vec)[:,1] if hasattr(model,"predict_proba") else None
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

==== MultinomialNB ====
Accuracy: 0.9722, Precision: 0.8642, Recall: 0.9396, F1: 0.9003

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.98       966
           1       0.86      0.94      0.90       149

    accuracy                           0.97      1115
   macro avg       0.93      0.96      0.94      1115
weighted avg       0.97      0.97      0.97      1115

Confusion Matrix:
 [[944  22]
 [  9 140]]
==== LogisticRegression ====
Accuracy: 0.9830, Precision: 0.9392, Recall: 0.9329, F1: 0.9360

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       966
           1       0.94      0.93      0.94       149

    accuracy                           0.98      1115
   macro avg       0.96      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115

Confusion Matrix:
 [[957   9]
 [ 10 139]]
==== RandomForest ====
Accuracy